<a href="https://colab.research.google.com/github/PadmaPujitaDasari/ChatBot/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# E-comerce Chatbot



## Necessary Imports

In [ ]:
import pandas as pd
import json
import nltk

# Load the dataset
with open('Ecommerce_FAQ_Chatbot_dataset.json', 'r') as file:
    dataset = json.load(file)

In [ ]:
greetings = [
    {"question": "Hello", "answer": "Hello! How can I assist you today?"},
    {"question": "Hi", "answer": "Hi there! How can I help you?"},
    {"question": "Hey", "answer": "Hey! What can I do for you today?"},
    {"question": "Good morning", "answer": "Good morning! How can I assist you?"},
    {"question": "Good afternoon", "answer": "Good afternoon! How can I help you?"},
    {"question": "Good evening", "answer": "Good evening! What can I do for you?"},
    {"question": "Thank you", "answer": "You're welcome! If you have any other questions, feel free to ask."},
    {"question": "Thanks", "answer": "No problem! Let me know if there's anything else I can help with."},
    {"question": "Goodbye", "answer": "Goodbye! Have a great day!"},
    {"question": "Bye", "answer": "Bye! Take care!"},
    {"question": "See you", "answer": "See you! Have a nice day!"},
    {"question": "Talk to you later", "answer": "Talk to you later! If you need anything, just ask."}
]

dataset['questions'].extend(greetings)

In [ ]:
df = pd.DataFrame(dataset['questions'])


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
df.head()

,question,answer
0,How can I create an account?,"To create an account, click on the 'Sign Up' b..."
1,What payment methods do you accept?,"We accept major credit cards, debit cards, and..."
2,How can I track my order?,You can track your order by logging into your ...
3,What is your return policy?,Our return policy allows you to return product...
4,Can I cancel my order?,You can cancel your order if it has not been s...


In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
!pip install pyspellchecker
from spellchecker import SpellChecker
spell = SpellChecker()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 20.4 MB/s eta 0:00:00


## Data Preprocessing

### Spell Checking and Correction

In [ ]:
def correct_spelling(text):
    tokens = word_tokenize(text.lower())
    corrected_tokens = [spell.correction(word) if spell.correction(word) else word for word in tokens]
    return ' '.join(corrected_tokens)

### Synonym Handling

In [ ]:
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return synonyms

### Tokenization

In [ ]:
def Tonekize(query):
    tokens = word_tokenize(query.lower())
    expanded_tokens = []
    for token in tokens:
      if token.isalnum() and token not in stop_words:
        synonyms = get_synonyms(token)
        if synonyms:
            expanded_tokens.extend(synonyms)
        else:
            expanded_tokens.append(token)
    return ' '.join(expanded_tokens)

### Preprocessing

In [ ]:
def preprocess(text):
    corrected_text = correct_spelling(text)
    tokenized_text = Tonekize(corrected_text)
    return tokenized_text

In [ ]:
df['question'] = df['question'].apply(preprocess)

In [ ]:
df.head()

,question,answer
0,produce create make history calculate chronicl...,"To create an account, click on the 'Sign Up' b..."
1,defrayment defrayal requital payment method_ac...,"We accept major credit cards, debit cards, and..."
2,caterpillar_tread rail chase track racecourse ...,You can track your order by logging into your ...
3,bring_back restitution repay regress turn_back...,Our return policy allows you to return product...
4,offset set_off invalidate scrub natural delete...,You can cancel your order if it has not been s...


##  Feature Extraction

In [ ]:
!pip install -Uq chromadb

In [ ]:
import chromadb
from chromadb.config import Settings
client = chromadb.Client(Settings())

In [ ]:
collection = client.create_collection(name="chat_collection")

In [ ]:
ids = []
documents = []
metadatas = []

for index, row in df.iterrows():
    ids.append(str(index))
    documents.append(row['question'])
    metadatas.append({"answer": row['answer']})

collection.add(
    ids=ids,
    documents=documents,
    metadatas=metadatas)

print("Embeddings stored successfully!")

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 54.0MiB/s]


Embeddings stored successfully!


In [ ]:
# client.delete_collection(name="chat_collection")
# collection.get()

## Model training

In [ ]:

def get_response(user_query):
    user_query = preprocess(user_query)
    results = collection.query(query_texts=user_query, n_results=1)

    # print(results['distances'][0][0], results['metadatas'][0][0]['answer'])

    if results:
        similarity = results['distances'][0][0]
        if similarity <= 1.1:
           return results['metadatas'][0][0]['answer']
        else:
           return "I'm sorry, I couldn't understand your question. Please contact our customer support team for further assistance."


## Interface

In [ ]:
def chat():
    print("Welcome to the E-commerce Chatbot! Type 'exit' to end the conversation.")
    end = False
    while end == False:
        user_query = input("You: ")
        if spell.correction(user_query.lower()) == 'exit':
            break
        response = get_response(user_query)

        for token in [ 'bye', 'goodbye', 'talk to you later','See you']:
            if token in word_tokenize(user_query.lower()):
              end = True
              break
        print("Bot:", response)


In [ ]:
chat()

Welcome to the E-commerce Chatbot! Type 'exit' to end the conversation.
You: heyy there,
Bot: Hey! What can I do for you today?
You: what payment tyes do you accept
Bot: We accept major credit cards, debit cards, and PayPal as payment methods for online orders.
You: payment types
Bot: We accept major credit cards, debit cards, and PayPal as payment methods for online orders.
You: payment methods
Bot: We accept major credit cards, debit cards, and PayPal as payment methods for online orders.
You: how can i produce anccount
Bot: To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process.
You: thanks
Bot: No problem! Let me know if there's anything else I can help with.
You: can i get a customer service
Bot: Installation services are available for select products. Please check the product description or contact our customer support team for more information and to request installation services